In [14]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
pd.options.display.max_rows = 999

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB 
from sklearn.naive_bayes import MultinomialNB 

In [2]:
filepath = '../datasets/news_cleaned_2018_02_13.csv'
nlinesfile = 9408908
nlinesrandomsample = 500_000
lines2skip = np.random.choice(np.arange(1,nlinesfile+1), (nlinesfile-nlinesrandomsample), replace=False)
df = pd.read_csv(filepath, skiprows=lines2skip)


df.shape

C:\Users\Ishan\Anaconda3\envs\nltktb\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,1,11,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(453569, 17)

In [3]:
df = df.drop(columns = ['authors', 'keywords', 'meta_description', 'tags', 'summary', 'source', 'id', 'Unnamed: 0', 'url', 'scraped_at', 'inserted_at', 'updated_at', 'meta_keywords'])

In [4]:
df.isna().mean()

domain     0.000000
type       0.047406
content    0.000002
title      0.008607
dtype: float64

In [5]:
df.dropna(inplace = True)

In [6]:
df = df[df['type'] != 'rumor']
df = df[df['type'] != 'unknown']
df = df[df['type'] != 'satire']

In [7]:
df['label'] = df['type'].map({'reliable' : 0, 'political' : 1, 'bias' : 2, 'fake' : 3, 'conspiracy' : 4, 'unreliable' : 5, 'clickbait' : 6, 'junksci' : 7, 'hate' : 8})

In [8]:
def tokenize(x):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(x)

df['tokens'] = df['title'].map(tokenize)
    
def lemmatize(x):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in x])

df['lemma'] = df['tokens'].map(lemmatize)

### Basic model

In [10]:
cvec = CountVectorizer(stop_words='english', ngram_range=(1, 2), max_df = .8, min_df = 3)
X = cvec.fit_transform(df['lemma'])



In [12]:
y = df['label']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42, stratify = y)

In [15]:
nb = MultinomialNB()

In [16]:
nb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [18]:
nb.score(X_train, y_train)

0.7112518497015696

In [19]:
nb.score(X_test, y_test)

0.6310322854715378